In [1]:
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
#from scripts.preprocess_cpv import process_cpv_data
from scripts.preprocess_categorielle import preprocess_categorielle, encoding_idAccordCadre
from scripts.preprocess_missing_values import clean_missing_values
from scripts.preprocess_cpv import add_cpv_hierarchy_column
from scripts.preprocess_numerique import log_transformation, scale_numerical_features

In [2]:
df = pd.read_csv('../data/train.csv', encoding='utf-8')

In [ ]:
columns_categorical = ['nature', 'procedure', 'formePrix', 'marcheInnovant', 'ccag',
                       'sousTraitanceDeclaree', 'typeGroupementOperateurs', 'origineFrance',
                       'idAccordCadre', 'codeCPV_2', 'tauxAvance_cat']
#dateNotification : pour les autres
#montant : target pour les autres
#codeCPV
columns_numerical = ['montant', 'dureeMois', 'offresRecues']

In [ ]:
#df = df.drop(df['dureeMois'].nlargest(2).index)

# Outliers


In [ ]:
# preprocess outliers

In [5]:
df = log_transformation(df, columns_numerical)

In [6]:
df[columns_numerical].describe()

,montant,dureeMois,offresRecues
count,239734.000000,239734.000000,239734.000000
mean,12.197424,3.025458,0.931331
std,1.654655,0.940933,0.851505
min,0.009950,0.000000,0.000000
25%,11.155840,2.564949,0.000000
50%,11.973875,3.218876,0.693147
75%,13.058107,3.891820,1.386294
max,32.236191,6.907755,9.507106


In [7]:
# ajouter le cas CPV avec le code de Ronan et Paul
df = add_cpv_hierarchy_column(df, cpv_column='codeCPV', level=2,
                             new_column_name=None)

In [8]:
cpv_group_counts = df['codeCPV_2'].value_counts()
top_n = 40
top_groups = cpv_group_counts.nlargest(top_n)

In [9]:
df = df[df['codeCPV_2'].isin(top_groups.index)]

df = df.drop(columns=['codeCPV'])

In [10]:
df


,nature,montant,procedure,dureeMois,formePrix,offresRecues,marcheInnovant,ccag,sousTraitanceDeclaree,typeGroupementOperateurs,tauxAvance,origineFrance,idAccordCadre,codeCPV_2
0,Marché,11.579346,Appel d'offres ouvert,3.178054,Unitaire,1.098612,0.0,Travaux,0.0,Pas de groupement,0.05,0.0,NaN,45000000
1,Marché,15.607270,Appel d'offres ouvert,3.610918,Mixte,0.000000,NaN,Pas de CCAG,NaN,Pas de groupement,0.00,0.0,NaN,45000000
2,Marché,14.603968,Appel d'offres ouvert,2.564949,Forfaitaire,1.098612,0.0,Travaux,0.0,Conjoint,NaN,NaN,2023-0202,45000000
3,Marché,14.690980,Appel d'offres ouvert,3.891820,Mixte,0.000000,NaN,Pas de CCAG,NaN,Pas de groupement,NaN,NaN,2023602099,71000000
4,Marché,14.634787,Procédure adaptée,3.610918,Unitaire,0.000000,NaN,Fournitures courantes et services,NaN,Pas de groupement,0.00,0.0,NaN,33000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239731,Marché,10.827766,Procédure adaptée,2.564949,Unitaire,0.000000,0.0,Pas de CCAG,0.0,Pas de groupement,NaN,NaN,2023S0156MAOOC,72000000
239732,Marché,12.996218,Appel d'offres ouvert,2.302585,Mixte,0.000000,NaN,Pas de CCAG,1.0,Pas de groupement,NaN,NaN,NaN,45000000
239733,Marché,11.254445,Procédure adaptée,2.833213,Unitaire,0.000000,NaN,Pas de CCAG,NaN,Pas de groupement,NaN,NaN,NaN,45000000
239734,Marché,12.264936,Procédure adaptée,3.218876,Forfaitaire,1.791759,0.0,Travaux,0.0,Conjoint,0.05,0.0,NaN,45000000


In [11]:
df = encoding_idAccordCadre(df)

In [12]:
df = clean_missing_values(df)
df.isna().sum()

Remaining missing values:
dureeMois                0
marcheInnovant           0
tauxAvance               0
offresRecues             0
sousTraitanceDeclaree    0
tauxAvance               0
origineFrance            0
dtype: int64


nature                      0
montant                     0
procedure                   0
dureeMois                   0
formePrix                   0
offresRecues                0
marcheInnovant              0
ccag                        0
sousTraitanceDeclaree       0
typeGroupementOperateurs    0
tauxAvance                  0
origineFrance               0
idAccordCadre               0
codeCPV_2                   0
dtype: int64

In [13]:
bins = [-0.001, 0.001, 0.05, 0.15, 1.0]
labels = ['no_advance', 'small_advance', 'medium_advance', 'large_advance']
df['tauxAvance_cat'] = pd.cut(df['tauxAvance'], bins=bins, labels=labels)

df = df.drop(columns=['tauxAvance'])
df['tauxAvance_cat'].value_counts()


tauxAvance_cat
no_advance        222507
small_advance       8537
medium_advance      3388
large_advance       2919
Name: count, dtype: int64

In [14]:
df = preprocess_categorielle(df, columns_categorical)

In [15]:
df

,montant,dureeMois,offresRecues,nature_Marché,nature_Marché de défense ou de sécurité,nature_Marché de partenariat,procedure_Appel d'offres ouvert,procedure_Appel d'offres restreint,procedure_Dialogue compétitif,procedure_Marché passé sans publicité ni mise en concurrence préalable,...,codeCPV_2_79000000,codeCPV_2_80000000,codeCPV_2_85000000,codeCPV_2_90000000,codeCPV_2_92000000,codeCPV_2_98000000,tauxAvance_cat_large_advance,tauxAvance_cat_medium_advance,tauxAvance_cat_no_advance,tauxAvance_cat_small_advance
0,11.579346,3.178054,1.098612,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,15.607270,3.610918,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,14.603968,2.564949,1.098612,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,14.690980,3.891820,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,14.634787,3.610918,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237346,10.827766,2.564949,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
237347,12.996218,2.302585,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
237348,11.254445,2.833213,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
237349,12.264936,3.218876,1.791759,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# scaling

In [16]:
#from sklearn.preprocessing import StandardScaler

#scaler = StandardScaler()
#df_scaled = df.copy()
#df_scaled[columns_numerical] = scaler.fit_transform(df[columns_numerical])
#
#df_scaled[columns_numerical].describe()

In [17]:
df = scale_numerical_features(df, columns_numerical)

In [18]:
df

,montant,dureeMois,offresRecues,nature_Marché,nature_Marché de défense ou de sécurité,nature_Marché de partenariat,procedure_Appel d'offres ouvert,procedure_Appel d'offres restreint,procedure_Dialogue compétitif,procedure_Marché passé sans publicité ni mise en concurrence préalable,...,codeCPV_2_79000000,codeCPV_2_80000000,codeCPV_2_85000000,codeCPV_2_90000000,codeCPV_2_92000000,codeCPV_2_98000000,tauxAvance_cat_large_advance,tauxAvance_cat_medium_advance,tauxAvance_cat_no_advance,tauxAvance_cat_small_advance
0,-0.374605,0.163837,0.191779,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2.060021,0.623807,-1.097563,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.453588,-0.487659,0.191779,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.506181,0.922299,-1.097563,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.472217,0.623807,-1.097563,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237346,-0.828888,-0.487659,-1.097563,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
237347,0.481805,-0.766452,-1.097563,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
237348,-0.570988,-0.202597,-1.097563,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
237349,0.039790,0.207215,1.005263,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
# I want to export in csv the df
df.to_csv('../data/train_preprocessed.csv', index=False, encoding='utf-8')

# Resume

In [ ]:
# Remove outliers
df = df.drop(df['dureeMois'].nlargest(2).index)

# Define column types
columns_categorical = ['nature', 'procedure', 'formePrix', 'marcheInnovant', 'ccag',
                      'sousTraitanceDeclaree', 'typeGroupementOperateurs', 'origineFrance',
                      'idAccordCadre', 'codeCPV_2', 'tauxAvance_cat']
columns_numerical = ['montant', 'dureeMois', 'offresRecues']

# Apply transformations
df = log_transformation(df, columns_numerical)
df = add_cpv_hierarchy_column(df, cpv_column='codeCPV', level=2)

# Filter top CPV groups
cpv_group_counts = df['codeCPV_2'].value_counts()
top_n = 40
top_groups = cpv_group_counts.nlargest(top_n)
df = df[df['codeCPV_2'].isin(top_groups.index)]
df = df.drop(columns=['codeCPV'])

# Handle ID fields and missing values
df = encoding_idAccordCadre(df)
df = clean_missing_values(df)

# Process tauxAvance as categorical
bins = [-0.001, 0.001, 0.05, 0.15, 1.0]
labels = ['no_advance', 'small_advance', 'medium_advance', 'large_advance']
df['tauxAvance_cat'] = pd.cut(df['tauxAvance'], bins=bins, labels=labels)
df = df.drop(columns=['tauxAvance'])

# One-hot encode categorical features
df = preprocess_categorielle(df, columns_categorical)

# Scale numerical features
df = scale_numerical_features(df, columns_numerical)